In [1]:
%load_ext autoreload
%autoreload 2

#load from local .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import sys
sys.path.append('..')

#standard libraries
import json
import os
import time
from typing import Any
from tqdm.notebook import tqdm

#external files
from src.preprocessor.preprocessing import FileIO
from src.database.weaviate_interface_v4 import WeaviateWCS, WeaviateIndexer
#weaviate
from weaviate.auth import AuthApiKey
from weaviate import connect_to_wcs
from weaviate.classes.config import Property
#misc
from rich import print
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor, as_completed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Constants

In [2]:
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']
collection_name = 'Huberman_minilm_256'

In [3]:
client = WeaviateWCS(endpoint=url, api_key=api_key)

/anaconda/envs/openai/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
client._client.is_connected()

True

In [5]:
%%time
response = client.vector_search('what is Tongkat Ali', collection_name, limit=5)

/anaconda/envs/openai/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


CPU times: user 604 ms, sys: 706 ms, total: 1.31 s
Wall time: 1.41 s


In [6]:
%%time
bm25 = client.keyword_search('what is the gut brain axis', collection_name, query_properties=['content'], limit=5)

CPU times: user 1.55 ms, sys: 514 µs, total: 2.06 ms
Wall time: 101 ms


In [7]:
%%time
hybrid = client.hybrid_search('what is the role of the brain', collection_name)

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 192 ms


In [29]:
docs = FileIO().load_parquet('../data/huberman_labs-minilm-256.parquet')

/anaconda/envs/openai/lib/python3.10/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


Shape of data: (23905, 13)
Memory Usage: 2.37+ MB


In [10]:
def convert_raw_data(raw_data: list[dict]) -> list[dict]:
    '''
    Converts raw YouTube json to correct format for 
    indexing on Weaviate. i.e. drops unused fields, 
    and coerces data types. 
    '''
    drops = ['channelId', 'isOwnerViewing', 'isCrawlable', 'allowRatings', \
             'author', 'isPrivate', 'isUnpluggedCorpus', 'isLiveContent']
    data = list(raw_data.values())
    for d in data:
        d['thumbnail_url'] = d['thumbnail']['thumbnails'][1].get('url')
        d['lengthSeconds'] = int(d['lengthSeconds'])
        d['viewCount'] = int(d['viewCount'])
        del d['thumbnail']
        for field in drops:
            del d[field]
    return data

In [32]:
for k in docs[0]:
    if k == 'view_count':
        print(f'({k} --> {type(docs[0][k])})')

(view_count --> <class 'int'>)

In [33]:
for d in docs:
    d['view_count'] = str(d['view_count'])

In [34]:
docs[0]['view_count']

'162689'

### Create Properties

In [15]:
from src.database.properties_template import properties
# from weaviate.classes.config import Tokenization, Property

In [17]:
print(properties)

[
    Property(
        name='video_id',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='title',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='length_seconds',
        dataType=<DataType.INT: 'int'>,
        description=None,
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='thumbnail_url',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=False,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='keywords',
        dataType=<DataType.TEXT_ARRAY: 'text[]'>,
        description=None,
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='short_description',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=False,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='view_count',
        dataType=<DataType.INT: 'int'>,
        description=None,
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='episode_num',
        dataType=<DataType.INT: 'int'>,
        description=None,
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='doc_id',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='content',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=False,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='guest',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='episode_url',
        dataType=<DataType.TEXT: 'text'>,
        description=None,
        indexFilterable=False,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    )
]

In [20]:
collection = client._client.collections.create(
    name='Delme_index',
    description='Collection of 193 episodes of the Huberman Labs podcast',
    properties=properties)

In [47]:
collection_name = 'Delme_index'

In [52]:
indexer = WeaviateIndexer(client)

In [24]:
indexer.create_collection(collection_name, properties, description='test index to delete')

Error creating collection, due to: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "Delme_index" already exists'}]}.


In [25]:
indexer._client.connect()
indexer._client.collections.exists(collection_name)

True

In [46]:
coll = client._client.collections.get('Delme_index')
client._client.connect()
coll.aggregate.over_all()

AggregateReturn(properties={}, total_count=0)

In [53]:
client._client.connect()
indexer.batch_index_data(docs[:10], collection_name)

 ... (more hidden) ...


Batch job completed in 0.01 minutes.


{'batch_object': <weaviate.collections.batch.collection._BatchCollection at 0x7faabcea1150>,
 'batch_errors': 10,
 'failed_objects': [ErrorObject(message="invalid integer property 'view_count' on class 'Delme_index': requires an integer, the given value is '162689'", object_=_BatchObject(collection='Delme_index', vector=[-0.022558197379112244, -0.021267101168632507, 0.05489972233772278, 0.07128095626831055, 0.06499787420034409, 0.05991670489311218, -0.01999964565038681, 0.0005336686153896153, 0.029837194830179214, -0.011097019538283348, -0.12040795385837555, 0.007165419403463602, -0.03563549369573593, 0.02375645749270916, -0.04856288060545921, -0.013060403987765312, -0.015637284144759178, -0.022509107366204262, 0.018875673413276672, 0.03482164070010185, 0.029338041320443153, 0.0023340671323239803, 0.13704490661621094, -0.007225443609058857, -0.0004594717756845057, 0.04137587547302246, 0.03369123488664627, -0.06239315867424011, -0.00755976140499115, -0.06433174014091492, -0.016501767560

In [8]:
questions = [
    "How can I optimize my sleep to improve cognitive function and overall well-being?",
    "What are some effective strategies for managing stress and anxiety based on neuroscience?",
    "Can you explain the science behind meditation and its benefits for mental health?",
    "How does nutrition impact brain health and cognitive performance?",
    "What are the best methods for enhancing focus and concentration?",
    "Can you discuss the relationship between exercise and brain function?",
    "How does exposure to natural light affect sleep quality and circadian rhythms?",
    "What are the most effective techniques for improving memory and learning?",
    "Can you explain the role of neurotransmitters in regulating mood and behavior?",
    "How does chronic stress impact brain structure and function?",
    "What are the implications of neuroplasticity for personal growth and development?",
    "How can we use breathing techniques to modulate our nervous system and reduce stress?",
    "Can you discuss the science of motivation and goal-setting from a neuroscience perspective?",
    "What are some effective ways to optimize brain health as we age?",
    "How does technology use affect brain function and mental well-being?",
    "Can you explain the effects of different types of music on the brain and mood?",
    "What are the benefits of exposure to nature for mental health and cognitive function?",
    "How can we cultivate resilience and adaptability in the face of challenges?",
    "Can you discuss the relationship between gut health and mental health?",
    "What are some practical strategies for improving emotional regulation and self-control?"
  ]

### Single Query: v4

In [9]:
start = time.perf_counter()
answer_v4 = client_v4.hybrid_search("What is serotonin good for", collection_name, return_properties=['content', 'title'])
end = time.perf_counter() - start
print(f'Python v4 client --> single query (n=1): {round(end,2)} seconds')

Python v4 client --> single query (n=1): 1.31 seconds

### Multi-Query (n=20) : v4

In [10]:
start = time.perf_counter()
answers_v4 = []
for q in questions:
    answers_v4.append(client_v4.hybrid_search(q, collection_name, return_properties=['content']))
end = time.perf_counter() - start
print(f'Python v4 client --> sequential queries (n=20): {round(end,2)} seconds')

Python v4 client --> sequential queries (n=20): 23.66 seconds

In [12]:
def main(query: str, collection_name: str, query_fields: list[str]=['content', 'title']):
    return client_v4.hybrid_search(query, collection_name, query_fields)

### Python Multithreading (n=20) : v4   --> Does not execute

In [13]:
start = time.perf_counter()
from warnings import filterwarnings
filterwarnings('ignore')

progress = tqdm(unit=": Queries", total=len(questions))
answers = []
try:
    with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as exec:
        futures = [exec.submit(main, q, collection_name) for q in questions]
        for future in as_completed(futures):
            answers.append(future.result()[0])
            progress.update(1)
    end = time.perf_counter() - start
    print(f'Python v4 client --> multithreading queries (n=20): {round(end,2)} seconds')

except Exception as e:
    print(f'ThreadPool did not execute due to {e}')

  0%|          | 0/20 [00:00<?, ?: Queries/s]

ThreadPool did not execute due to Query call with protocol GRPC search failed with message Channel closed!.

In [17]:
progress.close()

In [22]:
from llama_index.vector_stores import ChromaVectorStore